# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [9]:
import os
import glob
import psycopg2
import pandas as pd
import sql_queries as que
import create_tables as create

In [10]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
try:    
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
conn.set_session(autocommit=True)

In [11]:
# deletar tabelas
try: 
    cur.execute(que.song_table_drop)
    print("Sucess: Dropping song table")
except psycopg2.Error as e: 
    print("Error: Dropping table song")
    print (e)
    
try: 
    cur.execute(que.artist_table_drop)
    print("Sucess: Dropping artist table")
except psycopg2.Error as e: 
    print("Error: Dropping table artist")
    print (e) 
    
try: 
    cur.execute(que.songplay_table_drop)
    print("Sucess: Dropping songplay table")
except psycopg2.Error as e: 
    print("Error: Dropping table songplay")
    print (e) 
    
try: 
    cur.execute(que.user_table_drop)
    print("Sucess: Dropping user table")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e) 
    
try: 
    cur.execute(que.time_table_drop)
    print("Sucess: Dropping time table")
except psycopg2.Error as e: 
    print("Error: Dropping table user")
    print (e)     
    
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    
# criar tabelas
try:
    cur.execute(que.song_table_create)
    print("Sucess: Creating song table")
except psycopg2.Error as e: 
    print("Error: Creating song table")
    print (e)
    
try:
    cur.execute(que.artist_table_create)
    print("Sucess: Creating artists table")
except psycopg2.Error as e: 
    print("Error: Creating artists table")
    print (e) 
    
try:
    cur.execute(que.time_table_create)
    print("Sucess: Creating time table")
except psycopg2.Error as e: 
    print("Error: Creating time table")
    print (e)    
    
try:
    cur.execute(que.user_table_create)
    print("Sucess: Creating user table")
except psycopg2.Error as e: 
    print("Error: Creating user table")
    print (e)
    
try:
    cur.execute(que.songplay_table_create)
    print("Sucess: Creating songplay table")
except psycopg2.Error as e: 
    print("Error: Creating songplay table")
    print (e)      

Sucess: Dropping song table
Sucess: Dropping artist table
Sucess: Dropping songplay table
Sucess: Dropping user table
Sucess: Dropping time table
Sucess: Creating song table
Sucess: Creating artists table
Sucess: Creating time table
Sucess: Creating user table
Sucess: Creating songplay table


In [12]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [13]:
def getDataframeComplet(filepath):
    all_files = get_files(filepath=filepath)
    li = []

    for filename in all_files:
        df = pd.read_json(filename, lines=True)
        li.append(df)

    return pd.concat(li, axis=0, ignore_index=True)

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [14]:
# song_files = get_files("./data/song_data/")
# filePathSong = song_files[0]
# dfSongComplet = pd.read_json(filePathSong, lines=True)

In [15]:
dfSongComplet = getDataframeComplet("./data/song_data/")
dfSongComplet.rename(columns={'title': 'song'}, inplace=True)

In [16]:
#dfSongComplet

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [17]:
# método que adiciona colunas de IDs

# def addColumnId(dataFrameComplet, addIdColumn, nameColumnId, typeColumn):
#     # obter valores únicos
#     dfValuesUniques = dfSongComplet.drop_duplicates(subset=addIdColumn, keep='first')
    
#     # adicionando os ids 
#     dfValuesUniques[nameColumnId] = (dfValuesUniques.index + 1).astype(typeColumn)
    
#     # join para juntar este dataframe com o dataframe completo
#     dfValuesCompletWithId = dfValuesUniques.merge(dataFrameComplet, on=addIdColumn ,how='right', suffixes=('', '_dropp')).filter(regex="^(?!.*_dropp)")
    
#     return dfValuesCompletWithId

In [18]:
dfSongClear = dfSongComplet[['song_id', 'song', 'artist_id', 'year', 'duration']]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [19]:
# inserir cada linha do dataframe na tabela

def loopInsert(dfInsert, stringInsert):
    for index, row in dfInsert.iterrows():
        cur.execute(stringInsert, row)
        conn.commit()

In [20]:
loopInsert(dfSongClear, que.song_table_insert)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [21]:
artist_data = dfSongComplet[['artist_id', 'artist_name', 'artist_location', \
                             'artist_latitude', 'artist_longitude']]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [22]:
loopInsert(artist_data, que.artist_table_insert)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [23]:
# log_files = get_files("./data/log_data/")
# filePathLog = log_files[0]
# dfLogComplet = pd.read_json(filePathLog, lines=True)

dfLogComplet = getDataframeComplet("./data/log_data/")

In [24]:
# change columns name

dfLogComplet.rename(columns={'userId':'user_id'}, inplace=True)
dfLogComplet.rename(columns={'firstName':'first_name'}, inplace=True)
dfLogComplet.rename(columns={'lastName':'last_name'}, inplace=True)
dfLogComplet.rename(columns={'length':'duration'}, inplace=True)
dfLogComplet.rename(columns={'artist':'artist_name'}, inplace=True)

In [25]:
#dfLogComplet.head()

In [26]:
#dfLogComplet.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [27]:
dfLogComplet['date'] = pd.to_datetime(dfLogComplet['ts'], unit='ns')
dfLogComplet['start_time'] = [d.time() for d in dfLogComplet['date']]
dfLogComplet['hour'] = [d.time().hour for d in dfLogComplet['date']]
dfLogComplet['day'] = [d.date().day for d in dfLogComplet['date']]
dfLogComplet['week'] = [d.week for d in dfLogComplet['date']]
dfLogComplet['month'] = [d.month for d in dfLogComplet['date']]
dfLogComplet['year'] = [d.year for d in dfLogComplet['date']]
dfLogComplet['weekday'] = [d.weekday() for d in dfLogComplet['date']] 

In [28]:
#dfLogComplet.info()

In [29]:
time_data = dfLogComplet[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [30]:
loopInsert(time_data, que.time_table_insert)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [31]:
dfLogCompletClear = dfLogComplet[dfLogComplet['page']=='NextSong'].reset_index()
user_df = dfLogCompletClear[['user_id', 'first_name', 'last_name', 'gender', 'level']]

In [32]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6831 entries, 0 to 6830
Data columns (total 5 columns):
user_id       6831 non-null object
first_name    6831 non-null object
last_name     6831 non-null object
gender        6831 non-null object
level         6831 non-null object
dtypes: object(5)
memory usage: 266.9+ KB


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [33]:
loopInsert(user_df, que.user_table_insert)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [34]:
#dfSongComplet, dfLogComplet

In [35]:
#dfLogCompletClear.info()

In [36]:
# for index, row in df.iterrows():
for index, row in dfLogCompletClear.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(que.song_select, (row.song, row.artist_name, row.duration))
    #cur.execute(que.song_select, (row.song,))
    conn.commit()
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
        print(songid, artistid) 
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.user_id, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(que.songplay_table_insert, songplay_data)
    conn.commit()

AR5KOSW1187FB35FF4 Elena


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [37]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.